# BigQuery ML Model with Linear Feature Engineering/Transform

**Learning Objectives**

1. Create and evaluate linear model with BigQuery's `ML.FEATURE_CROSS`
1. Create and evaluate linear model with BigQuery's `ML.FEATURE_CROSS` and `ML.BUCKETIZE`
1. Create and evaluate linear model with `ML.TRANSFORM`


## Introduction 
In this notebook, we will create multiple linear models to predict the weight of a baby before it is born, using increasing levels of feature engineering using BigQuery ML. If you need a refresher, you can go back and look how we made a baseline model in the previous notebook [BQML Baseline Model](../solutions/3a_bqml_baseline.ipynb).

We will create and evaluate a linear model using BigQuery's `ML.FEATURE_CROSS`, create and evaluate a linear model using BigQuery's `ML.FEATURE_CROSS` and `ML.BUCKETIZE`, and create and evaluate a linear model using BigQuery's `ML.TRANSFORM`.

### Load necessary libraries

Check that the Google BigQuery library is installed and if not, install it. 

In [1]:
%%bash
sudo pip freeze | grep google-cloud-bigquery==1.6.1 || \
sudo pip install google-cloud-bigquery==1.6.1

google-cloud-bigquery==1.6.1


## Verify tables exist in BigQuery

Run the following cells to verify that we previously created the dataset and data tables. If not, go back to lab [1b_prepare_data_babyweight](../solutions/1b_prepare_data.ipynb) to create them.

In [2]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_train
LIMIT 0

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,mother_race


In [3]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_eval
LIMIT 0

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,mother_race


## Model 1:  Apply ML.FEATURE_CROSS

BigQuery ML now has `ML.FEATURE_CROSS`, a pre-processing clause that performs a feature cross with syntax `ML.FEATURE_CROSS(STRUCT(features), degree)` where features are comma-separated categorical columns and degree is highest degree of all combinations.

First, we'll create a BQML model using a feature cross.

In [4]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.model_1

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    L2_REG=0.1,
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ML.FEATURE_CROSS(
        STRUCT(
            is_male,
            plurality)
    ) AS gender_plurality_cross
FROM
    babyweight.babyweight_data_train

""


Let's retrieve the training statistics:

In [5]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL babyweight.model_1)

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.130173,None,None,28214


Once the model has trained, we'll evaluate its performance using `ML.EVALUATE`.

In [6]:
%%bigquery
SELECT
    *, SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.model_1,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks,
        ML.FEATURE_CROSS(
            STRUCT(
                is_male,
                plurality)
        ) AS gender_plurality_cross
    FROM
        babyweight.babyweight_data_eval
    ))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance,rmse
0,0.825543,1.129703,0.020253,0.670801,0.350758,0.35076,1.062875


## Model 2:  Apply `ML.BUCKETIZE`

Bucketize is a pre-processing function that creates "buckets" (e.g bins) - e.g. it bucketizes a continuous numerical feature into a string feature with bucket names as the value with syntax `ML.BUCKETIZE(feature, split_points)` with split_points being an array of numerical points to determine bucket bounds.

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.model_2

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    L2_REG=0.1,
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ML.FEATURE_CROSS(
        STRUCT(
            is_male,
            ML.BUCKETIZE(
                mother_age,
                GENERATE_ARRAY(15, 45, 1)
            ) AS bucketed_mothers_age,
            plurality,
            ML.BUCKETIZE(
                gestation_weeks,
                GENERATE_ARRAY(17, 47, 1)
            ) AS bucketed_gestation_weeks
        )
    ) AS crossed
FROM
    babyweight.babyweight_data_train

Executing query with job ID: 82df6b94-ad6c-4d3a-8a5c-80a4da77064c
Query executing: 75.39s

Let's now retrieve the training statistics and evaluate the model.

In [ ]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL babyweight.model_2)

We now evaluate our model on our eval dataset:

In [9]:
%%bigquery
SELECT
    *, SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.model_2,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks,
        ML.FEATURE_CROSS(
            STRUCT(
                is_male,
                ML.BUCKETIZE(
                    mother_age,
                    GENERATE_ARRAY(15, 45, 1)
                ) AS bucketed_mothers_age,
                plurality,
                ML.BUCKETIZE(
                    gestation_weeks,
                    GENERATE_ARRAY(17, 47, 1)
                ) AS bucketed_gestation_weeks
            )
        ) AS crossed
    FROM
        babyweight.babyweight_data_eval))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance,rmse
0,0.793926,1.047851,0.01763,0.646479,0.397799,0.3978,1.023646


Let's select the `mean_squared_error` from the evaluation table we just computed and square it to obtain the rmse.

## Model 3:  Apply `TRANSFORM`

Before we perform our prediction, we should encapsulate the entire feature set in a `TRANSFORM` clause. This way we can have the same transformations applied for training and prediction without modifying the queries.

Let's apply the TRANSFORM clause to the model_3 and run the query.

In [10]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.model_3

TRANSFORM(
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ML.FEATURE_CROSS(
        STRUCT(
            is_male,
            ML.BUCKETIZE(
                mother_age,
                GENERATE_ARRAY(15, 45, 1)
            ) AS bucketed_mothers_age,
            plurality,
            ML.BUCKETIZE(
                gestation_weeks,
                GENERATE_ARRAY(17, 47, 1)
            ) AS bucketed_gestation_weeks
        )
    ) AS crossed
)

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    L2_REG=0.1,
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    *
FROM
    babyweight.babyweight_data_train

""


Let's retrieve the training statistics:

In [11]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL babyweight.model_3)

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.047116,None,None,41691


We now evaluate our model on our eval dataset:

In [12]:
%%bigquery
SELECT
    *, SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.model_3,
    (
    SELECT
        *
    FROM
        babyweight.babyweight_data_eval
    ))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance,rmse
0,0.793926,1.047851,0.01763,0.646479,0.397799,0.3978,1.023646


Let's select the `mean_squared_error` from the evaluation table we just computed and square it to obtain the rmse.

## Lab Summary
In this lab, we created and evaluated a linear model using BigQuery's `ML.FEATURE_CROSS`, created and evaluated a linear model using BigQuery's `ML.BUCKETIZE`, and created and evaluated a linear model using BigQuery's `TRANSFORM`.

Copyright 2020 Google LLC
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.